# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
import os, glob
from macrohet import dataio, tile, visualise, notify

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
%%time
expt_ID = 'PS0000'

base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'

metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 11.9 s, sys: 1.16 s, total: 13.1 s
Wall time: 13.1 s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
Row Column                                              
3   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      INH          0.04            EC50
    9          WT      BDQ          0.02            EC50
4   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      INH          0.04            EC50
    9          WT      BDQ          0.02            EC50
5   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      INH             2            EC99
    9          WT      BDQ           2.5            EC99
6   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      INH             2            EC99
    9          WT      BDQ           2.5            EC99

In [90]:
acq_ID = row, column = (3, 5)

#### Define subset if non-square tiling or more than one contiguous region of images in imaging well. 

In [ ]:
# subset_field_IDs = ['1','6','7','8','11','12','13','14','15']

#### Load images using Zarr (WIP)

In [6]:
import zarr

In [91]:
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
zarr_group = zarr.open(image_dir, mode='r')

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [9]:
%%time
# image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
image_dir = os.path.join(base_dir, 'acquisition/Images')
dask_images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             # subset_field_IDs=['16', '17',  '20', '21'], 
                             # n_tile_rows = 2, n_tile_cols = 2,
                             set_plane='max_proj',
                             # set_channel=1,
                             # set_time = 1,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()
dask_images

CPU times: user 658 ms, sys: 3.96 ms, total: 662 ms
Wall time: 670 ms


dask.array<max-aggregate, shape=(75, 2, 6048, 6048), dtype=uint16, chunksize=(1, 2, 2160, 2160), chunktype=numpy.ndarray>

In [10]:
import numpy as np
import re
import pandas as pd

# Make some glimpses

In [7]:
# Define the pattern to match the file names
pattern = re.compile(r'\[(.*?)\]')


In [9]:
unique_IDs = [pattern.search(filename).group(1).replace("'", "") for filename in glob.glob('/mnt/SYNO/macrohet_syno/manuscript/dt_dev/*pdf')]


In [11]:

PS_sc_df_fn = '/mnt/SYNO/macrohet_syno/PS0000/results/preliminary_sc_measures/sc_dfs/sc_df_GT_70_area_false_outliers_removed.pkl'

sc_df = df = pd.read_pickle(PS_sc_df_fn)


In [12]:
from macrohet import visualise

In [18]:
import os
import re
import numpy as np
import cv2
import btrack
from tqdm.auto import tqdm
import zarr

In [26]:
segmentation=None
crop_size= side_length = 500
track_scale_factor=5.04
mask_outline=True

In [ ]:
seg_fn = ''`
image_dir = ''

In [23]:
# Sort the IDs based on their last two indices
sorted_ids = sorted(unique_IDs, key=lambda x: tuple(map(int, x.split('.')[-2:])))

# Print the sorted IDs
for id in sorted_ids:
    print(id)

100.3.4
137.3.4
193.3.4
2.3.4
207.3.4
227.3.4
230.3.4
241.3.4
284.3.4
324.3.4
344.3.4
346.3.4
349.3.4
358.3.4
36.3.4
385.3.4
391.3.4
401.3.4
41.3.4
426.3.4
435.3.4
472.3.4
545.3.4
547.3.4
552.3.4
563.3.4
575.3.4
596.3.4
621.3.4
75.3.4
831.3.4
1.3.5
117.3.5
134.3.5
150.3.5
194.3.5
197.3.5
201.3.5
218.3.5
252.3.5
254.3.5
278.3.5
289.3.5
302.3.5
319.3.5
330.3.5
337.3.5
339.3.5
341.3.5
352.3.5
355.3.5
363.3.5
366.3.5
367.3.5
380.3.5
393.3.5
414.3.5
438.3.5
44.3.5
450.3.5
477.3.5
481.3.5
486.3.5
503.3.5
507.3.5
510.3.5
513.3.5
515.3.5
528.3.5
530.3.5
539.3.5
542.3.5
544.3.5
564.3.5
586.3.5
601.3.5
610.3.5
624.3.5
670.3.5
802.3.5
810.3.5
823.3.5
869.3.5
893.3.5
351.3.6
358.3.6
361.3.6
405.3.6
458.3.6
509.3.6
593.3.6
610.3.6
628.3.6
728.3.6
755.3.6
964.3.6
123.3.7
147.3.7
188.3.7
196.3.7
199.3.7
219.3.7
257.3.7
271.3.7
287.3.7
295.3.7
307.3.7
318.3.7
323.3.7
337.3.7
378.3.7
443.3.7
48.3.7
487.3.7
526.3.7
551.3.7
558.3.7
567.3.7
672.3.7
83.3.7
88.3.7
95.3.7
1105.3.8
14.3.8
157.3.8
163.3.8
173.

In [53]:
seg_fn = ''

In [58]:
images.shape
loaded_image = str(acq_ID)

In [59]:
print(images.shape)

(75, 2, 6048, 6048)


In [ ]:
%%time

for unique_ID in tqdm(sorted_ids, total = len(unique_IDs)):
    acq_ID= row, column = int(unique_ID.split('.')[1]), int(unique_ID.split('.')[2]) 
    ID = int(unique_ID.split('.')[0])
    if str(acq_ID) not in image_dir:
        # print(f'Loading images for {unique_ID}')
        # image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
        # zarr_group = zarr.open(image_dir, mode='r')
        # images = (zarr_group.images[:,:,0,...])
        base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'
        loaded_image = str(acq_ID)
        image_dir = os.path.join(base_dir, 'acquisition/Images')
        images = tile.compile_mosaic(image_dir, 
                                     metadata, 
                                     row, column, 
                                     # subset_field_IDs=['16', '17',  '20', '21'], 
                                     # n_tile_rows = 2, n_tile_cols = 2,
                                     set_plane=1,
                                     # set_channel=1,
                                     # set_time = 1,
        #                             input_transforms = [input_transforms]
                                    ).compute().compute()
        images = images[:,:,0,...]
    rgb_stack = []
    if str(acq_ID) not in seg_fn:
        # load segmentation
        seg_fn = os.path.join(base_dir,f'labels/macrohet_seg_model/{acq_ID}.h5')
        with btrack.io.HDF5FileHandler(seg_fn,
                                       'r',
                                       obj_type='obj_type_1') as reader:
            segmentation = reader.segmentation
    # extract single cell df
    sc_df = df[df['Unique ID'] == unique_ID]
    
    time_values = sc_df['Time (hours)']
    for t in tqdm(time_values):
        t = int(t)
        sc_df_t = sc_df[sc_df['Time (hours)'] == t]
        # Extract xy coordinates and transpose for python and area from the cell information
        y_coord, x_coord, area, t = sc_df_t.loc[:, ['x', 'y', 'Mphi Area', 'Time (hours)']].values[0]
    
        # Scale according to tracking shrinkage
        y_coord, x_coord = y_coord * track_scale_factor, x_coord * track_scale_factor
    
        if not side_length:
            # Calculate the side length for cropping based on the square root of the area
            side_length = int(np.sqrt(area)) * 2
    
        # Calculate the cropping boundaries
        x_start = int(x_coord - side_length / 2)
        x_end = int(x_coord + side_length / 2)
        y_start = int(y_coord - side_length / 2)
        y_end = int(y_coord + side_length / 2)
        # Pad the boundaries if they exceed the image dimensions
        if x_start < 0:
            x_pad = abs(x_start)
            x_start = 0
        else:
            x_pad = 0
    
        if x_end > images.shape[2]:
            x_pad_end = x_end - images.shape[2]
            x_end = images.shape[2]
        else:
            x_pad_end = 0
    
        if y_start < 0:
            y_pad = abs(y_start)
            y_start = 0
        else:
            y_pad = 0
    
        if y_end > images.shape[3]:
            y_pad_end = y_end - images.shape[3]
            y_end = images.shape[3]
        else:
            y_pad_end = 0
    
        # Crop the image
        cropped_image = images[int(t), :, x_start:x_end, y_start:y_end]
    
        # Pad the cropped image if necessary
        cropped_image = np.pad(cropped_image, ((0, 0), (x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
    
        # extract the gfp and rfp channels to apply some vis techn
        gfp = cropped_image[0, ...]#.compute().compute()
        rfp = cropped_image[1, ...]#.compute().compute()
    
        # clip the images so that the contrast is more apparent
        contrast_lim_gfp = np.clip(gfp, 358, 5886)
        contrast_lim_rfp = np.clip(rfp, 480, 1300)
    
        norm_gfp = cv2.normalize(contrast_lim_gfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
        norm_rfp = cv2.normalize(contrast_lim_rfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    
        # put the modified gfp rfp back in place
        cropped_image[0, ...] = norm_gfp
        cropped_image[1, ...] = norm_rfp
    
        # Create an empty RGB image with the same shape as the input image
        rgb_image = np.zeros((cropped_image.shape[1], cropped_image.shape[2], 3), dtype=np.uint16)
    
        # Assign the first channel to the green channel of the RGB image
        rgb_image[:, :, 1] = cropped_image[0]
    
        # Assign the second channel to the red and blue channels of the RGB image to create magenta
        rgb_image[:, :, 0] = cropped_image[1]
        rgb_image[:, :, 2] = cropped_image[1]
    
        # scale down to 8bit
        rgb_image = np.uint8(rgb_image >> 8)
    
        if mask_outline:
            # load mask (singular)
            
            cropped_masks = segmentation[int(t), x_start:x_end, y_start:y_end]
            
            # Pad the cropped image if necessary
            cropped_masks = np.pad(cropped_masks, ((x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
    
            # extract only that segment
            seg_ID = cropped_masks[int(cropped_masks.shape[0] / 2), int(cropped_masks.shape[1] / 2)]
            print('here')
            if seg_ID == 0:
                instance_mask = np.zeros((crop_size, crop_size), dtype = np.uint8)
            else:
                instance_mask = (cropped_masks == seg_ID).astype(np.uint8)
    
            # draw outline
            contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(rgb_image, contours, -1, (0, 2 ** 8, 2 ** 8), thickness=2)  # make 8bit
    
        # downsize image to reduce storage demands
        rgb_image = cv2.resize(rgb_image, (rgb_image.shape[1] // 2, rgb_image.shape[0] // 2))
        rgb_stack.append(rgb_image)
        
    max_height = max(rgb_image.shape[0] for rgb_image in rgb_stack)
    max_width = max(rgb_image.shape[1] for rgb_image in rgb_stack)
    
    # Resize each image to the maximum dimensions
    resized_rgb_stack = [cv2.resize(rgb_image, (max_width, max_height)) for rgb_image in rgb_stack]
    
    # Stack the resized images
    rgb_stack = np.stack(resized_rgb_stack, axis=0)
    # Get the dimensions of the first frame
    height, width, _ = rgb_stack[0].shape
    
    # Define the frame rate (number of frames per second)
    frame_rate = len(rgb_stack) / 20  # Total frames divided by total seconds
    output_file = f"/mnt/SYNO/macrohet_syno/manuscript/dt_dev_glimpses/{unique_ID}_wmask.mp4"
    # Initialize VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, frame_rate, (width, height))
    
    # Write each frame to the video file
    for frame in rgb_stack:
        out.write(frame)
    
    # Release the VideoWriter object
    out.release()
    
    print(f"Video {unique_ID} saved successfully.")

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
Video 100.3.4 saved successfully.


  0%|          | 0/75 [00:00<?, ?it/s]

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
Video 137.3.4 saved successfully.


...

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
Video 181.4.9 saved successfully.


  0%|          | 0/75 [00:00<?, ?it/s]

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
Video 2.4.9 saved successfully.


  0%|          | 0/75 [00:00<?, ?it/s]

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
Video 200.4.9 saved successfully.


In [51]:
segmentation

In [30]:
viewer = napari.Viewer()

viewer.add_image(instance_mask)

<Image layer 'instance_mask' at 0x7fd846f475e0>

In [250]:
rgb_stack.shape

(75, 191, 191, 3)

In [203]:
from macrohet import notify

In [204]:
notify.send_sys_message()